In [1]:
# 이미지 경로 변환 - DB 직접 연동 버전
# type 컬럼으로 게시판 선택

import json
import os
import shutil
import re
from pathlib import Path
import pymysql
from sqlalchemy import create_engine

# ==================== DB 설정 ====================
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "crawling"

# ==================== 설정 영역 (여기를 수정하세요) ====================

# 게시판 타입 (필터링용)
BOARD_TYPE = "techInfo"  # ✨ 추가: hredu, cp, sales 등

# 이미지 원본 경로들 - 여러 경로 추가 가능
BASE_IMAGE_PATHS = [
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\namoeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\daumeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\namoeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor",
    # 더 추가하고 싶으면 여기에 계속 추가
]

# 이미지 복사본 저장 폴더 (타입별로 자동 생성)
OUTPUT_IMAGE_FOLDER = rf"C:\Users\LEEJUHWAN\Downloads\board_{BOARD_TYPE}_img"  # ✨ 타입 반영

# src에서 제거할 접두사 (이 부분을 제거하고 상대경로로 변경)
REMOVE_PREFIX = r"C:\Users\LEEJUHWAN\Downloads"

# ======================================================================


def find_image_folder(folder_name):
    """여러 경로에서 이미지 폴더 찾기"""
    for base_path in BASE_IMAGE_PATHS:
        image_folder = Path(base_path) / folder_name
        if image_folder.exists():
            return image_folder
    return None


def process_single_post(data, post_index):
    """하나의 post 데이터 처리"""
    
    # sourceId에서 문서ID 추출
    source_id = data.get('sourceId', '')
    if not source_id:
        print(f"⚠️ [{post_index}] sourceId 없음, 건너뜀")
        return data  # 원본 그대로 반환
    
    doc_id = source_id.lstrip('_')
    folder_name = f"brd{doc_id}"
    
    # 여러 경로에서 이미지 폴더 찾기
    image_folder = find_image_folder(folder_name)
    
    if not image_folder:
        print(f"⚠️ [{post_index}] 폴더 없음: {folder_name} (모든 경로 검색 완료)")
        return data  # 원본 그대로 반환
    
    # 복사본 저장할 문서별 폴더 생성
    output_doc_folder = Path(OUTPUT_IMAGE_FOLDER) / folder_name
    output_doc_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📄 [{post_index}] 문서ID: {doc_id}")
    print(f"   원본: {image_folder}")
    print(f"   복사: {output_doc_folder}")
    
    # HTML에서 img 태그 찾기
    html_content = data.get('content', '')
    if not html_content:
        print(f"   ⚠️ content 없음")
        return data
    
    img_pattern = r'<img[^>]+src="([^"]+)"[^>]*>'
    img_matches = list(re.finditer(img_pattern, html_content))
    
    if not img_matches:
        print(f"   ℹ️ 이미지 없음")
        return data
    
    print(f"   ✅ 이미지 {len(img_matches)}개 발견")
    
    # 파일 복사 & src 경로 변경
    new_html = html_content
    copied_count = 0
    
    for idx, match in enumerate(img_matches):
        old_src = match.group(1)
        
        # 원본 파일 (0, 1, 2...)
        old_file = image_folder / str(idx)
        
        # 복사본 파일 (.jpg 확장자 추가)
        new_file = output_doc_folder / f"{idx}.jpg"
        
        # 파일 복사
        if old_file.exists():
            shutil.copy2(old_file, new_file)
            copied_count += 1
            print(f"      ✅ {old_file.name} → {new_file.name}")
            
            # src 경로 변경 (상대경로로)
            new_src = str(new_file).replace('\\', '/')
            if REMOVE_PREFIX:
                prefix_with_slash = REMOVE_PREFIX.replace('\\', '/')
                if new_src.startswith(prefix_with_slash):
                    new_src = new_src[len(prefix_with_slash):].lstrip('/')
            
            new_html = new_html.replace(old_src, new_src, 1)
        else:
            print(f"      ⚠️ 파일 없음: {old_file.name} - src 공란 처리")
            # src를 빈 문자열로 변경
            new_html = new_html.replace(old_src, '', 1)
    
    print(f"   📊 복사 완료: {copied_count}/{len(img_matches)}개")
    
    # 수정된 HTML 적용
    data['content'] = new_html
    return data


def process_db(board_type):
    """DB에서 특정 type의 데이터만 읽고 처리 후 DB에 업데이트"""
    
    # 출력 폴더 생성
    os.makedirs(OUTPUT_IMAGE_FOLDER, exist_ok=True)
    
    # DB 연결
    print(f"📂 DB 연결 중...")
    print(f"📂 게시판 타입: {board_type}")  # ✨ 추가
    print(f"📂 검색할 이미지 경로: {len(BASE_IMAGE_PATHS)}개\n")
    
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASS,
        database=DB,
        port=PORT,
        charset='utf8mb4'
    )
    
    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:
            # 1. DB에서 특정 type의 데이터만 읽기
            query = "SELECT * FROM board_posts WHERE type = %s"  # ✨ WHERE 조건 추가
            cursor.execute(query, (board_type,))
            posts = cursor.fetchall()
            
            total_count = len(posts)
            print(f"✅ 총 {total_count}개의 posts 발견\n")
            
            # 2. 각 post 처리
            processed_count = 0
            
            for idx, post in enumerate(posts, 1):
                try:
                    # 원본 content 저장 (비교용)
                    original_content = post.get('content', '')
                    
                    # attaches_json을 파싱하여 attaches로 변환
                    if post.get('attaches_json'):
                        post['attaches'] = json.loads(post['attaches_json'])
                    else:
                        post['attaches'] = []
                    
                    # 기존 로직 그대로 사용
                    result = process_single_post(post, idx)
                    
                    # content가 변경된 경우만 DB 업데이트
                    if result['content'] != original_content:
                        update_query = """
                            UPDATE board_posts 
                            SET content = %s 
                            WHERE sourceId = %s AND type = %s
                        """  # ✨ WHERE 조건에 type도 추가
                        cursor.execute(update_query, (result['content'], result['sourceId'], board_type))
                        processed_count += 1
                    
                except Exception as e:
                    print(f"❌ [Post {idx}] 처리 오류: {e}")
                    continue
            
            # 변경사항 커밋
            connection.commit()
            
            print(f"\n{'='*60}")
            print(f"✅ 처리 완료!")
            print(f"   - 게시판 타입: {board_type}")  # ✨ 추가
            print(f"   - 처리된 posts: {processed_count}/{total_count}개")
            print(f"   - 복사된 이미지 위치: {OUTPUT_IMAGE_FOLDER}")
            print(f"   - DB 테이블: board_posts (content 컬럼 업데이트)")
            print(f"{'='*60}")
    
    finally:
        connection.close()
        print("\n✅ DB 연결 종료")


# 실행
if __name__ == "__main__":
    process_db(BOARD_TYPE)  # ✨ board_type 인자 추가

📂 DB 연결 중...
📂 게시판 타입: techInfo
📂 검색할 이미지 경로: 4개

✅ 총 185개의 posts 발견

⚠️ [1] 폴더 없음: brd4075480 (모든 경로 검색 완료)
⚠️ [2] 폴더 없음: brd3831102 (모든 경로 검색 완료)
⚠️ [3] 폴더 없음: brd3751452 (모든 경로 검색 완료)
⚠️ [4] 폴더 없음: brd3606302 (모든 경로 검색 완료)
⚠️ [5] 폴더 없음: brd3396654 (모든 경로 검색 완료)
⚠️ [6] 폴더 없음: brd2986156 (모든 경로 검색 완료)
⚠️ [7] 폴더 없음: brd2981863 (모든 경로 검색 완료)
⚠️ [8] 폴더 없음: brd2920676 (모든 경로 검색 완료)
⚠️ [9] 폴더 없음: brd2920576 (모든 경로 검색 완료)
⚠️ [10] 폴더 없음: brd2575319 (모든 경로 검색 완료)
⚠️ [11] 폴더 없음: brd2522281 (모든 경로 검색 완료)
⚠️ [12] 폴더 없음: brd2433315 (모든 경로 검색 완료)
⚠️ [13] 폴더 없음: brd2273572 (모든 경로 검색 완료)
⚠️ [14] 폴더 없음: brd2204464 (모든 경로 검색 완료)
⚠️ [15] 폴더 없음: brd2196550 (모든 경로 검색 완료)
⚠️ [16] 폴더 없음: brd2142231 (모든 경로 검색 완료)
⚠️ [17] 폴더 없음: brd2139874 (모든 경로 검색 완료)
⚠️ [18] 폴더 없음: brd2095884 (모든 경로 검색 완료)
⚠️ [19] 폴더 없음: brd2034861 (모든 경로 검색 완료)
⚠️ [20] 폴더 없음: brd2031402 (모든 경로 검색 완료)
⚠️ [21] 폴더 없음: brd2029308 (모든 경로 검색 완료)
⚠️ [22] 폴더 없음: brd2028973 (모든 경로 검색 완료)
⚠️ [23] 폴더 없음: brd1996811 (모든 경로 검색 완료)
⚠️ [24] 폴더 없음: brd1